###### Scraps

$$ a_i = \text{demand/weight/population at point } x_i $$

$$ p = \text{number of facilities to place}$$

### Sets + Vars

$$
I = \text{set of demand sites} \\
    J = \text{set of facility sites} \\
    S_i = \text{distance beyond which demand site }i\text{ is considered uncovered} \\
    d_{ij} = \text{distance from demand site }i\text{ to facility site }j \\
 a_i = \text{demand/weight/population at point } x_i  \\
 p = \text{number of facilities to place}
$$

 

- $N_i$ is the covering set of $i$
- the set of $j$ that cover demand point $i$
$$ N_i = \{ \, j \in J \, | \, d_{ij} \leq S_i \} $$

### Binary Varables

 \begin{equation}
  y_i=\left\{
  \begin{array}{@{}ll@{}}
    1 & \text{if demand site } i \text{ is NOT covered}\\
    0 & \text{otherwise}
  \end{array}\right.
\end{equation} 


 \begin{equation}
   x_j=\left\{
   \begin{array}{@{}11@{}}
     1 & \text{if facility is placed at site } j \\
     0 & \text{if not}
   \end{array}\right.
 \end{equation}

- number of facilities being added to the system as defined by $x_j$

 \begin{equation}
  e_j=\left\{
  \begin{array}{@{}ll@{}}
    1 & \text{if facility is NOT placed at site }j\\
    0 & \text{otherwise}
  \end{array}\right.
\end{equation} 


$$\sum_{i\in I} a_i = S $$

$$\sum_{j=0}^{J} x_j \leq  p $$

- minimize the amount of demand that is *not* covered (because $ y_i = 0 $ when it *is* covered)

$$\text{(o1)}\;\;\;\;  \min \sum_{i\in I} y_i \, a_i $$

- minimize whatever you need to add to the cover to cover everything

$$\text{(o2)}\;\;\;\; \min\sum_{j\in J} e_j $$

### Constraints

 $$\text{(c1)}\;\;\;\; \sum_{j\in N_i} x_j + y_i \geq 1 \text{, } \; \forall i \in I $$

$$\text{(c2)}\;\;\;\; \sum_{j\in J} x_j = P $$ 

** this is the P facility constraint. (o2) with (c1) and (c2) is basically the max cover problem **

$$\text{(c3)}\;\;\;\; \sum_{j\in N_i} x_j + e_j \geq 1 \text{, } \; \forall i \in I $$

 \begin{equation}
  \text{(c4)}\;\;\;\; x_j=\left\{
  \begin{array}{@{}ll@{}}
    0 & \\
    1 &
  \end{array}\right.
\end{equation} 

 \begin{equation}
  \text{(c5)}\;\;\;\; e_j=\left\{
  \begin{array}{@{}ll@{}}
    0 & \\
    1 &
  \end{array}\right.
\end{equation} 

 $$\text{(c6)}\;\;\;\; y_i \geq 0$$

 

#### Unspecified,  but Implied

$$ x_j + e_j \text{ covers completely } \forall i \in I $$

In [20]:
from pyscipopt import Model, multidict, quicksum

In [27]:
import networkx
import matplotlib.pyplot as P

In [5]:
#sample code below is from scipbook docs on Facility Location Problems
# http://scipbook.readthedocs.io/en/latest/flp.html

In [74]:
# specify the data
def make_data():
    I,d = multidict({0:12, 1:5, 2:6})            # demand
    J,M,f = multidict({0:[100, 0], 1:[100,0], 2:[100,0], 3:[100,0], 4:[100,0], 5:[100,0], 6:[100,0]}) # capacity, fixed costs
    N = multidict({0:[0,3], 1:[1,5], 2:[4,7]}) #in practice this would be generated according to distance
    c = {(0,0):4,  (0,1):6,  (0,2):9,    # distance from demand pt i to site j
         (1,0):5,  (1,1):0.7,  (1,2):7,
         (2,0):6,  (2,1):3,  (2,2):4,
         (3,0):0.5,  (3,1):5,  (3,2):3,
         (4,0):10, (4,1):8,  (4,2):0.86,
         (5,0):9, (5,1):0.8, (5,2):0.7,
         (6,0):2.5, (6,1):2.5, (6,2):1.3,
         }
    return I,J,d,M,f,N,c



In [75]:
# model --> algorithm : 

def flp(I,J,d,M,f,N,c):
    """flp -- model for the capacitated facility location problem
    Parameters:
        - I: set of customers
        - J: set of facilities
        - d[i]: demand for customer i
        - M[j]: capacity of facility j
        - f[j]: fixed cost for using a facility in point j
        - N[i]: set of j s.t. site j covers demand pt i
        - c[i,j]: unit cost of servicing demand point i from facility j (in our case, distance..
        )
    Returns a model, ready to be solved.
    """

    model = Model("flp")
    x,y,e = {},{},{}
    for i in I:
        y[i] = model.addVar(vtype="B", name="y(%s)"%i)
    for j in J:
        x[j] = model.addVar(vtype="B", name="x(%s)"%j)
        e[j] = model.addVar(vtype="B", name="e(%s)"%j)
    #for j in J:

    print len(N)
    print len(y)
    print len(I)
    for i in I:
        print i
        model.addCons(x[i]+quicksum(y[j] for j in N[i]) >= 1, "Coverage Constraint(%s)"%i)

    #we'll set p=2 here
    p=1
    model.addCons(quicksum(x[j] for j in x) == p, "P-facility constraint(%s)"%p)

    for i in I:
        model.addCons(y[i] >= 0, "LB on Y_i (%s)"%i)
        #model.addCons(x[i,j] <= d[i]*y[j], "Strong(%s,%s)"%(i,j))

    model.setObjective(
        quicksum(y[i]*d[i] for i in I),
        #quicksum(f[j]*y[j] for j in J) +
        #quicksum(c[i,j]*x[i,j] for i in I for j in J),
        "maximize")
    model.data = x,y

    return model


In [76]:
if __name__ == "__main__":
    I,J,d,M,f,N,c = make_data()
    model = flp(I,J,d,M,f,N,c)
    model.hideOutput(quiet=False) #verbose mode
    model.optimize()

    EPS = 1.e-6
    x,y = model.data
    edges = [j for j in x if model.getVal(x[j]) > EPS]
    facilities = [i for i in y if model.getVal(y[i]) > EPS]

    print("Optimal value:", model.getObjVal())
    print("Facilities at nodes:", facilities)
    print("Edges:", edges)

    try: # plot the result using networkx and matplotlib
        import networkx as NX
        import matplotlib.pyplot as P
        P.clf()
        G = NX.Graph()

        other = [i for i in y if i not in facilities]
        customers = ["c%s"%j for j in d]
        G.add_nodes_from(facilities)
        G.add_nodes_from(other)
        G.add_nodes_from(customers)
        for (i,j) in edges:
            G.add_edge("c%s"%i,j)

        position = NX.drawing.layout.spring_layout(G)
        NX.draw(G,position,node_color="y",nodelist=facilities)
        NX.draw(G,position,node_color="g",nodelist=other)
        NX.draw(G,position,node_color="b",nodelist=customers)
        P.show()
    except ImportError:
        print("install 'networkx' and 'matplotlib' for plotting")

3
3
3
0
1
2
('Optimal value:', 23.0)
('Facilities at nodes:', [0, 1, 2])
('Edges:', [0])


TypeError: 'int' object is not iterable

<Figure size 432x288 with 0 Axes>

In [66]:
for i in I:
    print "Coverage Constraint "+str(i)
    print str(x[i])+str(quicksum(y[j] for j in N[i]))+ ">= 1"


Coverage Constraint 0
x(0)Expr({Term(): 0.0, Term(y(0)): 1.0, Term(y(2)): 1.0, Term(y(1)): 1.0})>= 1
Coverage Constraint 1
x(1)Expr({Term(): 0.0, Term(y(0)): 1.0, Term(y(2)): 1.0, Term(y(1)): 1.0})>= 1
Coverage Constraint 2
x(2)Expr({Term(): 0.0, Term(y(0)): 1.0, Term(y(2)): 1.0, Term(y(1)): 1.0})>= 1


In [67]:
print x

{0: x(0), 1: x(1), 2: x(2), 3: x(3), 4: x(4), 5: x(5), 6: x(6)}


In [29]:
from math import sqrt

0.8660254037844386